In [2]:
'''The proposed algorithm detects changes between 2 satellite images using
   Principle Component Analysis (PCA) and K-Means Clustering.

   Implementation is based on this scientific paper:
       Turgay Celik
       "Unsupervised Change Detection in Satellite Images Using Principal Component Analysis
       and k-Means Clustering"
       IEEE GEOSCIENCE AND REMOTE SENSING LETTERS, VOL. 6, NO. 4, OCTOBER 2009 
       DOI: 10.1109/LGRS.2009.2025059
       and the Matlab implementation on GitHub by leduckhai:
       https://github.com/leduckhai/Change-Detection-PCA-KMeans


   The following codes are implemented only for PERSONAL USE, e.g improving
   programming skills in the domain of Image Processing and Computer Vision.
   If you use this algorithm, please cite the paper mentioned above to support
   the authors.

   Parameters:
       image1: the 1st input image, should be in RGB-scale
       image2: the 2nd input image, should be in RGB-scale
       h: the block size, h is an integer
       rate: coefficient of thresholding'''

import torch
import numpy as np
import matplotlib.pyplot as plt

def ChangeDetection_PCA_KMeans(image1, image2, h, rate): # manually adding two images
    # Set input parameters
    if image1.shape != image2.shape:
        print('Image 1 and 2 must be in the same size')
        return
    rows, cols, channels = image1.shape
    if channels != 3:
        image1 = np.repeat(image1, 3, axis=2) / 255
        image2 = np.repeat(image2, 3, axis=2) / 255
    fig, axs = plt.subplots(2, 2)
    axs[0, 0].imshow(image1)
    axs[0, 0].set_title('Original image 1')
    axs[0, 1].imshow(image2)
    axs[0, 1].set_title('Original image 2')
    
    # Calculate difference image (Equation 1)
    dif_image = np.abs(image1.astype(float) - image2.astype(float))
    axs[1, 0].imshow(dif_image)
    axs[1, 0].set_title('Difference image')
    
    # Padding for the difference image
    pad = np.zeros((rows+h-1, cols+h-1, channels))
    pad[:rows, :cols, :] = dif_image
    
    # Divide the difference image into h x h non-overlapping blocks (Equation 2)
    vector_set = np.zeros((rows*cols, 3*h*h))
    count = 0
    for i in range(rows):
        for j in range(cols):
            block = pad[i:i+h, j:j+h, :]
            vector_set[count, :] = np.reshape(block, (1, -1))
            count += 1
    
    # PCA algorithm
    # Calculate average vector of the set (Equation 3)
    mean_vector = np.mean(vector_set, axis=0)
    dif_vector = vector_set - mean_vector
    # Calculate covariance matrix (Equation 4)
    covar = np.cov(dif_vector, rowvar=False)
    # Compute eigenvectors and eigenvalues of the covariance matrix
    eigvalue_mat, eigvector = np.linalg.eig(covar)
    eigvalue = np.diag(eigvalue_mat)
    # Sort the eigen vectors according to the descending eigenvalues
    index = np.argsort(-eigvalue)
    eigvalue = eigvalue[index]
    eigvector = np.abs(eigvector[:,index])
    # Choose number of eigenvectors satisfied the required magnitude percentage
    EigenvectorPer = 1
    lim = np.ceil(EigenvectorPer*eigvector.shape[1]).astype(int)
    eigvector = eigvector[:, :lim]
    # Choose number of eigenvectors satisfied the thresholding
    for k1 in range(len(eigvalue)-1, -1, -1):
        if np.sum(eigvalue[k1:]) >= rate * np.sum(eigvalue):
            break
    eigvector = eigvector[:, k1:]
    # Project difference image vector set onto eigenvector space to get feature vector space
    feature = torch.Tensor(vector_set) @ torch.Tensor(eigvector)
    
    # K-Means algorithm
    label = torch.kmeans(feature, 2).labels
    change_map = np.reshape(label, (rows, cols)).T - 1
    axs[1, 1].imshow(change_map)
    axs[1, 1].set_title('Change map')
    plt.show()
    return
